In [3]:
import pandas as pd
import numpy as np
import os as os
import openpyxl as op


In [4]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# IMPORTACIÓN DE DATOS

## Fuentes de los datos:

>la DIAN proporciona información en sus estadisticas sobre Recaudo Bruto por Seccionales y Tipo de Impuesto 2005 - 2025, estos datos están en millones de $COP 

>El DANE en su Encuesta Anual Manufacturera recolecta datos de las empresas con el fin de obtener información básica del sector fabril que permita el conocimiento de su estructura, evolución y desarrollo. Utilizaremos esta base de datos para darnos acceso a la información sobre las empresas del sector industrial en particular del pago de impuestos.

## Acceso a datos:
>DIAN: https://www.dian.gov.co/dian/cifras/Paginas/EstadisticasRecaudo.aspx

>DANE: portal micro datos: https://microdatos.dane.gov.co/index.php/catalog/central/about

In [ ]:
# Mostrar el directorio actual 
directorio_inicial = os.getcwd()
print(f"Directorio inicial: {directorio_inicial}")

## Importación de dataset de la DIAN:
El objetivo es obtener datos para el intervalo 2018-2023 que estén completos y limpios con una nueva columna llamada departamento que será la que usemos para unir los datos del dane con la DIAN.

In [5]:
#como queremos seguir en el directorio actual, pero los datos están cargados en otro folder, 
# vamos a generar una función que nos permita acceder a esa ruta donde estás los datasets y 
# ahorrar espacio el tiempo en el codigo, cosa que nos será util más adelante al importar 
# más datos, porque ya tendremos la ruta al folder definida.

# Ruta base de la carpeta "datasets"
ruta_datasets = r"C:\Users\Kayak\OneDrive\Escritorio\U\2025-2\IA_aplicada\proyecto\datasets" 
# Nombre del primer archivo que es tipo xlsx
Recaudo_Bruto = "Estadisticas-de-Recaudo-bruto-por-seccionales-y-tipo-de-impuesto-2005-2025.xlsx"

#creamos ruta para importar el priemer archivo
ruta_Recaudo_Bruto = os.path.join(ruta_datasets, Recaudo_Bruto)

# carga del archivo usando la ruta, saltandonos las primeras 5 filas y las últimas 10 porque son titulos o aclaraciones 
df_impuestos = pd.read_excel(ruta_Recaudo_Bruto,  skiprows=5,skipfooter=10)
#veamos que tipo de datos hay en cada columna y convirtamos a float los datos del recaudo que sean números.
# al ver el dataframe, notamos que la fila 1 y 2 están fusionadas por lo que debemos eliminar una dado que no aporta nada al analisis.
df_impuestos=df_impuestos.drop(index=0)

#de otro lado la columna AÑO debe ser rellenada, porque tiene muchos valores NaN debido a tener celdas combinadas en la base de datos.
# para ello usamos el método fillna con el parámetro method='ffill' (forward fill) que rellena los valores NaN con el último valor no nulo encontrado hacia adelante.
df_impuestos['AÑO'] = df_impuestos['AÑO'].ffill()

#Ahora bien, vamos a manejar datos solamente de 2018 a 2023, por lo que filtramos el dataframe para quedarnos solo con esos años.
df_impuestos['AÑO'] = pd.to_numeric(df_impuestos['AÑO'], errors='coerce')  # Convertir a numérico, forzando errores a NaN
df_impuestos = df_impuestos[(df_impuestos['AÑO'] >= 2018) & (df_impuestos['AÑO'] <= 2023)]
df_impuestos['AÑO'] = df_impuestos['AÑO'].astype(int)
df_impuestos.reset_index(inplace=True, drop=True) 

# Normalizar los nombres en la columna DIRECCIÓN_SECCIONAL
df_impuestos["DIRECCIÓN SECCIONAL"] = (
    df_impuestos["DIRECCIÓN SECCIONAL"]
    .str.strip()  # elimina espacios antes/después
    .str.upper() ) # pasa todo a mayúsculas

# los datos de la DIAN están en millones $COP, los del Dane están en miles de $COP
# por lo tanto, para que las unidades sean consistentes, convertiremos los datos de la DIAN a miles de $COP
#hay columnas tipo texto asi que vamos a ignorarlas para trabajar sin errores en una conversión de formato.
columnas_excluidas = ['AÑO', 'DIRECCIÓN SECCIONAL']
columnas_a_convertir = df_impuestos.columns.drop(columnas_excluidas)

# conversión solo a esas columnas
#dado que hay valores que no son número sino guiones, 
#usaremos .apply(pd.to_numeric, errors='coerce') para reemplazar automáticamente con NaN (Not a Number) 
#esos valores con guión, permitiendo que el resto de la conversión continúe.

df_impuestos[columnas_a_convertir] = df_impuestos[columnas_a_convertir].apply(pd.to_numeric, errors='coerce')
df_impuestos[columnas_a_convertir] = df_impuestos[columnas_a_convertir] * 1000
#convertimos los valores NaN a 0  
df_impuestos = df_impuestos.fillna(0)


In [7]:

# Diccionario ciudad -> departamento (ajustado a mayúsculas)
ciudades_departamentos = {
    "ADUANAS BARRANQUILLA": "ATLÁNTICO",
    "ADUANAS BOGOTA": "CUNDINAMARCA",
    "ADUANAS CALI": "VALLE DEL CAUCA",
    "ADUANAS CARTAGENA": "BOLÍVAR",
    "ADUANAS DE CUCUTA": "NORTE DE SANTANDER",
    "ADUANAS DE MEDELLIN": "ANTIOQUIA",
    "IMPUESTOS BOGOTA": "CUNDINAMARCA",
    "GRANDES CONTRIBUYENTES BOGOTA": "BOGOTÁ, D.C.",
    "OPERERATIVA DE GRANDES CONTRIBUYENTES": "BOGOTÁ, D.C.",
    "OPERATIVA GRANDES CONTRIBUYENTES": "BOGOTÁ, D.C.",

    "ARAUCA": "ARAUCA",
    "ARMENIA": "QUINDÍO",
    "BARRANCABERMEJA": "SANTANDER",
    "BARRANQUILLA": "ATLÁNTICO",
    "BUCARAMANGA": "SANTANDER",
    "BUENAVENTURA": "VALLE DEL CAUCA",
    "CALI": "VALLE DEL CAUCA",
    "CARTAGENA": "BOLÍVAR",
    "CUCUTA": "NORTE DE SANTANDER",
    "FLORENCIA": "CAQUETÁ",
    "GIRARDOT": "CUNDINAMARCA",
    "IBAGUE": "TOLIMA",
    "INIRIDA": "GUAINÍA",
    "IPIALES": "NARIÑO",
    "LETICIA": "AMAZONAS",
    "MAICAO": "LA GUAJIRA",
    "MANIZALES": "CALDAS",
    "MEDELLIN": "ANTIOQUIA",
    "MITU": "VAUPÉS",
    "MONTERIA": "CÓRDOBA",
    "NEIVA": "HUILA",
    "PALMIRA": "VALLE DEL CAUCA",
    "PASTO": "NARIÑO",
    "PEREIRA": "RISARALDA",
    "POPAYAN": "CAUCA",
    "PUERTO ASIS": "PUTUMAYO",
    "PUERTO CARREÑO": "VICHADA",
    "QUIBDO": "CHOCÓ",
    "RIOHACHA": "LA GUAJIRA",
    "SAN ANDRES": "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA",
    "SAN JOSE DEL GUAVIARE": "GUAVIARE",
    "SANTA MARTA": "MAGDALENA",
    "SINCELEJO": "SUCRE",
    "SOGAMOSO": "BOYACÁ",
    "TULUA": "VALLE DEL CAUCA",
    "TUNJA": "BOYACÁ",
    "URABA": "ANTIOQUIA",
    "VALLEDUPAR": "CESAR",
    "VILLAVICENCIO": "META",
    "YOPAL": "CASANARE"
}

# Diccionario departamento -> código
departamento_codigos = {
    "ANTIOQUIA": 5,
    "ATLÁNTICO": 8,
    "BOGOTÁ, D.C.": 11,
    "BOLÍVAR": 13,
    "BOYACÁ": 15,
    "CALDAS": 17,
    "CAQUETÁ": 18,
    "CAUCA": 19,
    "CESAR": 20,
    "CÓRDOBA": 23,
    "CUNDINAMARCA": 25,
    "CHOCÓ": 27,
    "HUILA": 41,
    "LA GUAJIRA": 44,
    "MAGDALENA": 47,
    "META": 50,
    "NARIÑO": 52,
    "NORTE DE SANTANDER": 54,
    "QUINDÍO": 63,
    "RISARALDA": 66,
    "SANTANDER": 68,
    "SUCRE": 70,
    "TOLIMA": 73,
    "VALLE DEL CAUCA": 76,
    "ARAUCA": 81,
    "CASANARE": 85,
    "PUTUMAYO": 86,
    "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA": 88,
    "AMAZONAS": 91,
    "GUAINÍA": 94,
    "GUAVIARE": 95,
    "VAUPÉS": 97,
    "VICHADA": 99
}

#eliminación de filas los totales por año para poder unir las bases y generar columna dpto.
df_impuestos_arreglado = df_impuestos[~df_impuestos["DIRECCIÓN SECCIONAL"].str.contains("TOTAL AÑO", case=False, na=False)].copy()
# Asignación de departamento
df_impuestos_arreglado["departamento"] = df_impuestos_arreglado["DIRECCIÓN SECCIONAL"].map(ciudades_departamentos)

# 4. Asignación de  código de departamento
df_impuestos_arreglado["dpto"] = df_impuestos_arreglado["departamento"].map(departamento_codigos)
df_impuestos_arreglado['dpto'] = df_impuestos_arreglado['dpto'].apply(pd.to_numeric, errors='coerce')
df_impuestos_arreglado = df_impuestos_arreglado.dropna(subset=["dpto"])
df_impuestos_arreglado["dpto"] = df_impuestos_arreglado["dpto"].astype(int)

In [9]:
df_impuestos_arreglado

,AÑO,DIRECCIÓN SECCIONAL,RENTA CUOTAS,IVA DECLARACIONES 2/,RETENCIONES 3/,EXTERNOS 4/,SEGURIDAD DEMOCRATICA,PATRIMONIO,RIQUEZA,PRECIOS DE TRANSFERENCIA,GMF 5/,CONSUMO 6/,GASOLINA Y ACPM,CREE RETENCIONES,CREE DECLARACIONES,AL CARBONO,SIMPLE 7/\n,NORMALIZACION 8/,CONSUMO BIENES 9/,IVA_EXTERIOR,PRODUCTOS PLÁSTICOS 10/,BEBIDAS ULTRA PROCESADAS 11/,PRODUCTOS COMESTIBLES 12/,IMPUESTO ESPECIAL PARA EL CATATUMBO14/,POR CLASIFICAR 13/,TOTAL **,departamento,dpto
0,2018,ADUANAS BARRANQUILLA,0.00,0.00,0.00,"1,873,414,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,873,414,000.00",ATLÁNTICO,8
1,2018,ADUANAS BOGOTA,0.00,0.00,0.00,"5,085,685,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,085,685,000.00",CUNDINAMARCA,25
2,2018,ADUANAS CALI,0.00,0.00,0.00,"942,667,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"942,667,000.00",VALLE DEL CAUCA,76
3,2018,ADUANAS CARTAGENA,0.00,0.00,0.00,"5,458,611,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,458,611,000.00",BOLÍVAR,13
4,2018,ADUANAS DE CUCUTA,0.00,0.00,0.00,"20,399,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"20,399,000.00",NORTE DE SANTANDER,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,2023,TUNJA,"84,015,041.48","98,277,380.99","183,799,938.92",0.00,0.00,"1,576,510.91","44,488.00",0.00,"655,737.00","16,815,650.67",0.00,"60,030.00","19,272.65",0.00,"20,975,235.00",0.00,"13,313.00",301.00,0.00,0.00,0.00,0.00,"1,070,350.40","407,323,250.02",BOYACÁ,15
289,2023,URABA,0.00,0.00,0.00,"41,779,583.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"41,779,583.00",ANTIOQUIA,5
290,2023,VALLEDUPAR,"118,943,430.61","128,540,778.23","270,395,968.14","29,084,869.00",0.00,"5,290,057.00","174,856.84",0.00,"26,940.00","8,167,874.44",0.00,"82,839.00","47,106.67",0.00,"15,409,979.00",0.00,"143,019.00","1,181.00",0.00,0.00,0.00,0.00,"1,387,967.40","577,696,866.34",CESAR,20
291,2023,VILLAVICENCIO,"192,430,238.05","277,405,986.23","423,170,839.06",0.00,0.00,"7,527,912.00","326,712.51",0.00,"385,923.00","28,189,741.00",671.00,"58,587.00","89,250.01",0.00,"21,260,018.00",0.00,"231,952.00","1,083.00",0.00,0.00,0.00,0.00,"1,737,660.17","952,816,573.02",META,50


In [ ]:
"""

columnas_a_sumar = df_impuestos_arreglado.columns.drop(['AÑO', 'DIRECCIÓN SECCIONAL', 'departamento'])

# Agrupa por 'dpto' y SÓLO suma las columnas de la lista 'columnas_a_sumar'
df_impuestos_agrupado = df_impuestos_arreglado.groupby('dpto')[columnas_a_sumar].sum()
"""

In [ ]:
df_impuestos_arreglado.info(   )

## Importación de dataset del DANE: Encuesta Anual Manufacturera

In [ ]:
#indicación de la ruta de los datasets y el separador de los archivos csv
ruta_datasets = r'C:\Users\Kayak\OneDrive\Escritorio\U\2025-2\IA_aplicada\proyecto\datasets'
CSV_SEPARATOR = ';' 

# Diccionario de configuración de archivos y formato exacto por año
archivos = {
    2018: {'nombre': 'EAM_ANONIMIZADA_2018.csv', 'formato': 'csv'},
    2019: {'nombre': 'EAM_ANONIMIZADA_2019.csv', 'formato': 'csv'},
    2020: {'nombre': 'EAM_ANONIMIZADA_2020.xlsx', 'formato': 'xlsx'}, 
    2021: {'nombre': 'EAM_ANONIMIZADA_2021.xlsx', 'formato': 'xlsx'}, 
    2022: {'nombre': 'EAM_ANONIMIZADA_2022.xlsx', 'formato': 'xlsx'},
    2023: {'nombre': 'EAM_ANONIMIZADA_2023.xlsx', 'formato': 'xlsx'}
}

# Diccionario de decodificación y selección de columnas
columnas_renombre = {
    'dpto': 'DEPARTAMENTO',
    'periodo': 'AÑO',
    'C3R20C3': 'Impuesto de Renta para la Equidad',
    'c3r25c3': 'Impuesto del 4 x 1 000',
    'c3r37c3': 'Impuesto de Industria y Comercio',
    'c3r38c3': 'Impuesto Predial y sobre Vehículos'
}
columnas_seleccion = list(columnas_renombre.keys()) # Las columnas originales que buscamos
columnas_impuestos = list(columnas_renombre.values())[2:] # Solo los nombres de los impuestos finales

# Lista para almacenar los DataFrames procesados anualmente
list_df_anuales = []

# --- 2. Bucle de Procesamiento ---
for año, dataset in archivos.items():
    nombre_archivo = dataset['nombre']
    formato_archivo = dataset['formato']
    ruta_archivo = os.path.join(ruta_datasets, nombre_archivo)
    df_EAM = None

    # A. Importación Condicional
    try:
        if formato_archivo == 'xlsx':
            df_EAM = pd.read_excel(ruta_archivo) 
        elif formato_archivo == 'csv':
            df_EAM = pd.read_csv(ruta_archivo, sep=CSV_SEPARATOR)
        else:
            print(f" Error: Formato '{formato_archivo}' no reconocido.")
            continue
        
        # Estandarización de nombres: convierte los nombres de columnas a minúsculas
        # Esto soluciona la mayoría de los KeyErrors si el archivo no respeta mayúsculas/minúsculas.
        df_EAM.columns = df_EAM.columns.str.lower().str.strip()
        print(f"✅ Importado y nombres estandarizados: {nombre_archivo}")
        
    except FileNotFoundError:
        print(f" Error: Archivo NO encontrado en la ruta para el año {año}.")
        continue
    except Exception as e:
        print(f" Error al leer el archivo {nombre_archivo}. Detalles: {e}")
        continue

    # B. Selección y Renombramiento (Idéntico a tu referencia, pero con nombres limpios)
    # 1. Aseguramos que la selección se haga con los nombres limpios (minúsculas)
    columnas_a_seleccionar_limpias = [col.lower() for col in columnas_seleccion]
    
    try:
        # Aquí se aplica la selección de columnas.
        df_EAM_util = df_EAM[columnas_a_seleccionar_limpias].copy()
    except KeyError as e:
        print(f" ERROR CRÍTICO en el año {año}: Columnas no encontradas. Faltan: {e}. Revisar el archivo.")
        print(f"Columnas disponibles: {df_EAM.columns.tolist()}")
        continue

    # 2. Renombramos usando los nombres limpios como clave
    columnas_renombre_limpio = {k.lower(): v for k, v in columnas_renombre.items()}
    df_EAM_util = df_EAM_util.rename(columns=columnas_renombre_limpio)
    

    # C. Conversión de Tipos de Datos (Necesario para sumar después de leer CSV/TXT)
    for col in columnas_impuestos:
        df_EAM_util[col] = pd.to_numeric(df_EAM_util[col], errors='coerce')

    # D. Agrupación y Suma por Departamento (Igual que en tu referencia)
    df_EAM_sumado_por_dpto = df_EAM_util.groupby('DEPARTAMENTO').sum(numeric_only=True).reset_index()

    # E. Corrección del Campo 'AÑO'
    df_EAM_sumado_por_dpto['AÑO'] = año

    # F. Almacenamiento para concatenación
    list_df_anuales.append(df_EAM_sumado_por_dpto)


# --- 3. Consolidación Final ---
# ¡Concatenación vertical (apilamiento) de todos los resúmenes anuales!
df_EAM_consolidado = pd.concat(list_df_anuales, ignore_index=True)

print("\n--- PROCESO TERMINADO ---")
print(f"DataFrames consolidados para los años: {min(archivos.keys())} a {max(archivos.keys())}")
print(f"Total de registros consolidados (Filas): {len(df_EAM_consolidado)}")
print("\nPrimeras filas del DataFrame Consolidado:")
print(df_EAM_consolidado.head(10))

Iniciando procesamiento del año 2018 (CSV)...


C:\Users\Kayak\AppData\Local\Temp\ipykernel_19028\1580666563.py:48: DtypeWarning: Columns (302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363) have mixed types. Specify dtype option on import or set low_memory=False.
  df_EAM = pd.read_csv(ruta_archivo, sep=CSV_SEPARATOR)


✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2018.csv
Iniciando procesamiento del año 2019 (CSV)...


C:\Users\Kayak\AppData\Local\Temp\ipykernel_19028\1580666563.py:48: DtypeWarning: Columns (302) have mixed types. Specify dtype option on import or set low_memory=False.
  df_EAM = pd.read_csv(ruta_archivo, sep=CSV_SEPARATOR)


✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2019.csv
Iniciando procesamiento del año 2020 (XLSX)...
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2020.xlsx
Iniciando procesamiento del año 2021 (XLSX)...
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2021.xlsx
Iniciando procesamiento del año 2022 (XLSX)...
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2022.xlsx
Iniciando procesamiento del año 2023 (XLSX)...
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2023.xlsx

--- PROCESO TERMINADO ---


NameError: name 'config_archivos' is not defined

In [12]:
df_EAM_consolidado.rename(columns={'DEPARTAMENTO': 'dpto'}, inplace=True)

In [13]:
df_EAM_consolidado

,dpto,AÑO,Impuesto de Renta para la Equidad,Impuesto del 4 x 1 000,Impuesto de Industria y Comercio,Impuesto Predial y sobre Vehículos
0,5,2018,0.00,"161,143,552.00","209,177,913.00","63,446,694.00"
1,8,2018,0.00,"45,410,108.00","61,443,599.00","17,974,027.00"
2,11,2018,0.00,"147,122,946.00","288,579,378.00","91,316,705.00"
3,13,2018,0.00,"42,862,190.00","108,368,051.00","34,589,274.00"
4,15,2018,0.00,"8,914,892.00","14,440,007.00","1,554,821.00"
...,...,...,...,...,...,...
133,70,2023,0.00,"874,167.00","978,190.00","132,388.00"
134,73,2023,0.00,"10,627,912.00","20,467,192.00","1,638,427.00"
135,76,2023,0.00,"209,375,068.00","412,867,650.00","77,196,312.00"
136,85,2023,0.00,"6,942,318.00","14,735,260.00","405,911.00"


In [ ]:
df_EAM_consolidado

,dpto,AÑO,Impuesto de Renta para la Equidad,Impuesto del 4 x 1 000,Impuesto de Industria y Comercio,Impuesto Predial y sobre Vehículos
0,5,2018,0.00,"161,143,552.00","209,177,913.00","63,446,694.00"
23,5,2019,0.00,"151,994,122.00","220,700,716.00","71,751,426.00"
46,5,2020,"482,650.00","156,406,257.00","215,804,911.00","60,555,115.00"
69,5,2021,0.00,"193,311,710.00","298,198,694.00","58,628,406.00"
92,5,2022,0.00,"241,099,797.00","452,491,859.00","74,554,417.00"
...,...,...,...,...,...,...
45,99,2019,0.00,"116,583.00","385,992.00","411,570.00"
68,99,2020,0.00,"148,656.00","312,613.00","636,542.00"
91,99,2021,0.00,"152,997.00","263,707.00","425,200.00"
114,99,2022,0.00,"159,521.00","308,701.00","310,645.00"


In [14]:

df_EAM_impuestos = pd.merge(df_EAM_consolidado, df_impuestos_arreglado, on='dpto', how='outer')

In [16]:
df_EAM_impuestos.head(20)

,dpto,AÑO_x,Impuesto de Renta para la Equidad,Impuesto del 4 x 1 000,Impuesto de Industria y Comercio,Impuesto Predial y sobre Vehículos,AÑO_y,DIRECCIÓN SECCIONAL,RENTA CUOTAS,IVA DECLARACIONES 2/,RETENCIONES 3/,EXTERNOS 4/,SEGURIDAD DEMOCRATICA,PATRIMONIO,RIQUEZA,PRECIOS DE TRANSFERENCIA,GMF 5/,CONSUMO 6/,GASOLINA Y ACPM,CREE RETENCIONES,CREE DECLARACIONES,AL CARBONO,SIMPLE 7/\n,NORMALIZACION 8/,CONSUMO BIENES 9/,IVA_EXTERIOR,PRODUCTOS PLÁSTICOS 10/,BEBIDAS ULTRA PROCESADAS 11/,PRODUCTOS COMESTIBLES 12/,IMPUESTO ESPECIAL PARA EL CATATUMBO14/,POR CLASIFICAR 13/,TOTAL **,departamento
0,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2018,ADUANAS DE MEDELLIN,0.00,0.00,0.00,"820,001,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"820,001,000.00",ANTIOQUIA
1,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2018,MEDELLIN,"2,190,594,000.00","5,272,050,000.00","8,485,847,000.00",0.00,"5,000.00","2,130,000.00","66,023,000.00","164,000.00","2,924,428,000.00","319,487,000.00",0.00,"1,487,000.00","6,755,000.00","5,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,895,000.00","19,274,870,000.00",ANTIOQUIA
2,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2018,URABA,0.00,0.00,0.00,"52,673,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"52,673,000.00",ANTIOQUIA
3,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2019,ADUANAS DE MEDELLIN,0.00,0.00,0.00,"930,191,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"930,191,000.00",ANTIOQUIA
4,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2019,MEDELLIN,"2,048,459,000.00","5,801,992,000.00","9,498,548,000.00",0.00,"1,010,000.00","134,271,000.00","6,965,000.00","488,000.00","3,381,637,000.00","361,522,000.00","2,000.00","538,000.00","6,784,000.00","4,000.00","13,171,000.00","190,547,000.00","13,774,000.00",0.00,0.00,0.00,0.00,0.00,"11,400,000.00","21,471,113,000.00",ANTIOQUIA
5,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2019,URABA,0.00,0.00,0.00,"71,572,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"71,572,000.00",ANTIOQUIA
6,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2020,ADUANAS DE MEDELLIN,0.00,0.00,0.00,"897,215,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"897,215,000.00",ANTIOQUIA
7,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2020,MEDELLIN,"2,073,955,000.00","5,738,992,000.00","9,483,689,000.00",0.00,"47,000.00","151,975,000.00","4,892,000.00","420,000.00","3,339,786,000.00","228,818,000.00","45,000.00","847,000.00","3,288,000.00","3,000.00","54,096,000.00","88,693,000.00","627,000.00","11,000.00",0.00,0.00,0.00,0.00,"13,616,000.00","21,183,802,000.00",ANTIOQUIA
8,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2020,URABA,0.00,0.00,0.00,"67,753,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"67,753,000.00",ANTIOQUIA
9,5,"2,018.00",0.00,"161,143,552.00","209,177,913.00","63,446,694.00",2021,ADUANAS DE MEDELLIN,0.00,0.00,0.00,"1,223,070,917.33",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,223,070,917.33",ANTIOQUIA
